# Deriving MODIS Data from GEE

The notebook is focused on slicing chuncks of MODIS data on GEE and exporting it as .tif files to the Google Drive.

In [6]:
import numpy as np
import pandas as pd
import geopandas as gpd

import time
from datetime import datetime, timedelta
from os import listdir
from os.path import join

import ee
from utils_modis import extract_MODIS_chunk, bands, mod_names

In [7]:
gdf = gpd.read_file('../data_v02/gadm41_USA_1.json')
states = ['Minnesota', 'Iowa', 'Wisconsin', 'NorthDakota', 'SouthDakota', 'Nebraska', 'Illinois', 'Indiana', 'Missouri', 'Michigan', 'Kansas']
gdf = gdf[gdf.NAME_1.isin(states)].reset_index(drop=True)
bbox = np.round(gdf.union_all().bounds)

upper_midwest = ee.Geometry.Rectangle(list(bbox))

In [8]:
def mask(image):
    qa = image.select(bands[-1])
    
    cloud = qa.bitwiseAnd(1 << 0).eq(1)
    shadow = qa.bitwiseAnd(1 << 1).eq(1)
    cirrus = qa.bitwiseAnd(1 << 2).eq(1)
    
    # Cloud or shadow or cirrus = 1, else 0
    cloud_mask = cloud.Or(shadow).Or(cirrus).rename(mod_names[-1]).toInt16()

    image = image.select(bands[:-1]).addBands(cloud_mask)
    return image
    
def create_daily_export(date):
    ee_date = ee.Date(date.strftime('%Y-%m-%d'))
   
    daily_modis = (ee.ImageCollection('MODIS/061/MOD09GA')
        .filterBounds(upper_midwest)
        .filterDate(ee_date, ee_date.advance(1, 'day'))
    )

    daily_image = daily_modis.first().select(bands[:-1])
    #daily_image_masked = mask(daily_image)
    task = ee.batch.Export.image.toDrive(image=daily_image, description=f'MODIS_{date.year}_{date.day_of_year}', scale=500, 
                                      region=upper_midwest, maxPixels=1e10, folder=None)
   
    return task

In [ ]:
start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2020-12-31')

current_date = start_date
tasks = []
t = time.time()

'''Download MODIS without the cloud band'''
while current_date <= end_date:
    task = create_daily_export(current_date)
    task.start()
    tasks.append(task)
    
    if len(tasks) >= 10:
        tasks = [t for t in tasks if t.status()['state'] not in ['COMPLETED', 'FAILED']]
        time.sleep(10)
   
    current_date += timedelta(days=1)
    
print(f"All tasks submitted! Time: {round((time.time()-t)/60,3)} min")